In [1]:
""" 
# Expected format is below

Start Configuration

2 1 6
4 * 8
7 5 3

Goal Configuration

1 2 3
8 * 4
7 6 5

"""

import re
start_configuration = []
goal_configuration = []

# File Reading
# Disclaimer: Potential repetition of digits not guarded against
with open('astar_in.txt', 'r') as file:
    lines = file.readlines()
    for l in lines:
        if l.strip() == 'Start Configuration':
            read_start = True
            read_goal = False
        if l.strip() == 'Goal Configuration':
            read_start = False
            read_goal = True
        

        if re.match(r'(\d|\*) (\d|\*) (\d|\*)', l.strip()):
            arr = l.strip().split(' ')
            if read_start:
                start_configuration.append(arr)
            elif read_goal:
                goal_configuration.append(arr)
    file.close()
    
print(start_configuration)
print(goal_configuration)

[['2', '1', '6'], ['4', '*', '8'], ['7', '5', '3']]
[['1', '2', '3'], ['8', '*', '4'], ['7', '6', '5']]


In [ ]:

def is_equal(config_a, config_b) -> bool:
    """
    Checks if 2 configurations are equal
    """
    for i in range(len(config_a)):
        for j in range(len(config_b)):
            if config_a[i][j] != config_b[i][j]:
                return False
    return True


def display_node(node) -> None:
    """
    Node display function
    
    :param node: At all times, must contain config, g_value, h_value, and f_value
      or this will raise an exception.
    """
    print('g(n) = ', node['g_value'])
    print('h(n) = ', node['h_value'])
    print('f(n) = ', node['f_value'])
    
    if node.get('P_value', None):
        print('P(n) = ', node['P_value'])
    
    if node.get('S_value', None):
        print('S(n) = ', node['S_value'])

    for r in node['config']:
        print(*r)


def display_solution(solution_traversal) -> None:
    """
    Displays traversal of solution given
    an array of nodes that the solution consists of.
    """
    print("==========")
    print("|SOLUTION|")
    print("==========")
    print()

    for n in solution_traversal:
        display_node(n)
        print()


def count_misplaced_tiles_heuristic_3x3(config) -> int:
    """
    Returns number of tiles in the wrong position.
    """
    goal = goal_configuration
    misplaced_count = 0

    for i in range(0, len(goal)):
        for j in range(0, len(goal[i])):
            if config[i][j] != '*' and goal[i][j] != config[i][j]:
                misplaced_count += 1

    return misplaced_count


def get_manhattan_distance_heuristic_3x3(config) -> int:
    """
    Calculates number of absolute vertical and horizontal moves to convert
    current config to the goal_config.
    """
    # TODO (jace): goal_state_coordinates must be based on a goal state
    # instead of being hardcoded
    goal_state_coordinates = {
        '1': (0, 0),
        '2': (0, 1),
        '3': (0, 2),
        '4': (1, 2),
        '5': (2, 2),
        '6': (2, 1),
        '7': (2, 0),
        '8': (1, 0),
    }
    manhattan_distance = 0

    for i in range(len(config)):
        for j in range(len(config)):
            if config[i][j] == '*':
                continue
            true_position = goal_state_coordinates[config[i][j]]
            manhattan_distance += abs(i - true_position[0]) + abs(j - true_position[1])

    return manhattan_distance


def get_nilsson_sequence_score(config) -> tuple([int, int, int]):
    """
    Returns a tuple consisting of the total f_score based on
    the nilsson sequence score method, the manhattan distance (or P)
    and the sequence score (or S)
    """
    clockwise_expectation = {str(i + 1): str((i + 1) % 8 + 1) for i in range(8)}
    manhattan_distance = get_manhattan_distance_heuristic_3x3(config)
    sequence_score = 0

    for i in range(len(config)):
        for j in range(len(config)):
            if config[i][j] == '*':
                continue
            elif i == 1 and j == 1:
                sequence_score += 1
            else:
                if i == 0 and j < 2 and config[i][j + 1] != clockwise_expectation[config[i][j]]:
                    sequence_score += 2
                elif j == 2 and i < 2 and config[i + 1][j] != clockwise_expectation[config[i][j]]:
                    sequence_score += 2
                elif j > 0 and i == 2 and config[i][j - 1] != clockwise_expectation[config[i][j]]:
                    sequence_score += 2
                elif i > 0 and j == 0 and config[i - 1][j] != clockwise_expectation[config[i][j]]:
                    sequence_score += 2
                    
    nilsson_sequence_score = manhattan_distance + 3 * sequence_score

    return nilsson_sequence_score, manhattan_distance, sequence_score


def goal_check(config) -> bool:
    """
    Returns True if there are no misplaced tiles.
    
    Simply makes use of the misplaced tiles heuristics if there are any
    tiles out of place.
    """
    return count_misplaced_tiles_heuristic_3x3(config) == 0


def determine_empty_space_coordinates(config) -> tuple([int, int]):
    """
    Determines x, y coordinates of the '*'
    in a config.
    """
    for i in range(0, len(config)):
        for j in range(0, len(config[i])):
            if config[i][j] == '*':
                return i, j


def expand_3x3(config) -> list:
    """
    Retrieve the possible actions from the current state.
    """
    import copy
    i, j = determine_empty_space_coordinates(config)
    expansion_list = []
    
    if i > 0:
        expansion = copy.deepcopy(config)
        expansion[i][j], expansion[i-1][j] = expansion[i-1][j], expansion[i][j]
        expansion_list.append(expansion)

    if i < 2:
        expansion = copy.deepcopy(config)
        expansion[i][j], expansion[i+1][j] = expansion[i+1][j], expansion[i][j]
        expansion_list.append(expansion)

    if j > 0:
        expansion = copy.deepcopy(config)
        expansion[i][j], expansion[i][j-1] = expansion[i][j-1], expansion[i][j]
        expansion_list.append(expansion)

    if j < 2:
        expansion = copy.deepcopy(config)
        expansion[i][j], expansion[i][j+1] = expansion[i][j+1], expansion[i][j]
        expansion_list.append(expansion)

    return expansion_list


def retrieve_config_with_least_f_value(open_list) -> list:
    """
    Returns a list of indices on the open_list
    that correspond to the configs with best
    f-scores.
    """
    import math

    min_val = math.inf
    min_indices = []

    for i in range(0, len(open_list)):
        if open_list[i]['f_value'] <= min_val:
            min_val = open_list[i]['f_value']

    for i in range(0, len(open_list)):
        if open_list[i]['f_value'] == min_val:
            min_indices.append(i)
        
    return min_indices


def solution_traversal(leaf_node) -> list:
    """
    Returns a list consisting of the solution
    path.
    """
    traversal_list = []
    curr_pointer = leaf_node

    while curr_pointer != None:
        traversal_list.append(curr_pointer)
        curr_pointer = curr_pointer['parent']

    traversal_list.reverse()
    return traversal_list


def solve_8_puzzle_with_a_star(start_node, heuristic_fn, is_manhattan=False, is_nilsson=False) -> list:
    """
    Performs A-Star search given a start config and a
    heuristic function.
    
    Additionally, allows storage of P-values and S-values if the
    Manhattan and Nilsson heuristics are used.
    """
    
    # (1) Put the start node s on a list called OPEN and compute f(s)
    open_list = [{
        'config': start_node,
        'g_value': 0,
        'h_value': (h := heuristic_fn(start_node)),
        'f_value': 0 + (h if not is_nilsson else h[0]),
        'parent': None
    }]
    closed_list = []
    search_cost = 0
    while len(open_list) > 0:
        # (2) if open is empty, exit with failure; otherwise continue
        if len(open_list) == 0:
            raise Exception('Failure')

        # (3) Remove from OPEN that node whose f value is smallest
        node_indices_with_least_f_value = retrieve_config_with_least_f_value(open_list)

        # (resolve ties for for minimal f values arbitrarily but always in favor of any
        # goal node.)
        for i in node_indices_with_least_f_value:
            if goal_check(open_list[i]['config']):
                return solution_traversal(open_list[i]), search_cost
            
        node_index_with_least_f_value = node_indices_with_least_f_value[0]
        current_state = open_list[node_index_with_least_f_value]

        # and put it on a list called CLOSED. Call this node n.
        closed_list.append(current_state)
        display_node(current_state)
        
        # (4) If n is a goal node, exit with the solution path obtained by tracing back
        # the pointers; otherwise continue.
        if goal_check(current_state['config']):
            return solution_traversal(current_state), search_cost

        # (removed current node from open)
        open_list = open_list[:node_index_with_least_f_value] + open_list[node_index_with_least_f_value + 1:]

        
        # (5) Expand node n, generating all of its successors. If there are no successors, go immediately
        # to 2. For each successor n_i, compute f(n_i)
        expansions = [{
            'config': x,
            'heuristic_temp': (heuristic_temp := heuristic_fn(x)),
            'g_value': (g := current_state['g_value'] + 1),
            'h_value': (h := heuristic_temp if not is_nilsson else heuristic_temp[0]),
            'f_value': g + (h if not is_nilsson else heuristic_temp[0]),
            'P_value': (heuristic_temp[1] if is_nilsson else h if is_manhattan else None),
            'S_value': (heuristic_temp[2] if is_nilsson else None),
            # (6) Associate with the successors not already on either OPEN or CLOSED the f values just computed.
            'parent': current_state
        } for x in expand_3x3(current_state['config'])]
        search_cost += len(expansions)
        
        # (7) Associate with those successors that were already on OPEN or CLOSED
        for expansion in expansions:
            is_found_on_closed_list = False
            is_found_on_open_list = False
            for item in closed_list:
                if is_equal(expansion['config'], item['config']):
                    is_found_on_closed_list = True
                    # ...the smaller of the f-values just computed and their previous f values.
                    if item['f_value'] > expansion['f_value']:
                        item['g_value'] = expansion['g_value']
                        item['h_value'] = expansion['h_value']
                        item['f_value'] = expansion['f_value']
                        # Put on OPEN those successors on CLOSED whose f values were thus lowered, and
                        # redirect to n the pointers from all nodes whose f values were lowered.
                        open_list.append(expansion)

            for item in open_list:
                if is_equal(expansion['config'], item['config']):
                    is_found_on_open_list = True
                    # ...the smaller of the f-values just computed and their previous f values.
                    if item['f_value'] > expansion['f_value']:
                        item['g_value'] = expansion['g_value']
                        item['h_value'] = expansion['h_value']
                        item['f_value'] = expansion['f_value']

            # (6 cont.) Put these nodes on OPEN and direct pointers from them back to n
            if not is_found_on_open_list and not is_found_on_closed_list:
                open_list.append(expansion) 

In [3]:
heuristic_function = count_misplaced_tiles_heuristic_3x3
solution, search_cost = solve_8_puzzle_with_a_star(start_configuration, heuristic_function)

g(n) =  0
h(n) =  7
f(n) =  7
2 1 6
4 * 8
7 5 3
g(n) =  1
h(n) =  7
f(n) =  8
2 * 6
4 1 8
7 5 3
g(n) =  1
h(n) =  7
f(n) =  8
2 1 6
4 5 8
7 * 3
g(n) =  1
h(n) =  7
f(n) =  8
2 1 6
* 4 8
7 5 3
g(n) =  1
h(n) =  7
f(n) =  8
2 1 6
4 8 *
7 5 3
g(n) =  2
h(n) =  6
f(n) =  8
* 2 6
4 1 8
7 5 3
g(n) =  2
h(n) =  7
f(n) =  9
2 6 *
4 1 8
7 5 3
g(n) =  2
h(n) =  7
f(n) =  9
2 1 6
4 5 8
7 3 *
g(n) =  2
h(n) =  7
f(n) =  9
* 1 6
2 4 8
7 5 3
g(n) =  2
h(n) =  7
f(n) =  9
2 1 *
4 8 6
7 5 3
g(n) =  2
h(n) =  7
f(n) =  9
2 1 6
4 8 3
7 5 *
g(n) =  3
h(n) =  6
f(n) =  9
4 2 6
* 1 8
7 5 3
g(n) =  3
h(n) =  6
f(n) =  9
1 * 6
2 4 8
7 5 3
g(n) =  3
h(n) =  6
f(n) =  9
2 1 6
4 8 3
7 * 5
g(n) =  2
h(n) =  8
f(n) =  10
2 1 6
4 5 8
* 7 3
g(n) =  2
h(n) =  8
f(n) =  10
2 1 6
7 4 8
* 5 3
g(n) =  3
h(n) =  7
f(n) =  10
2 6 8
4 1 *
7 5 3
g(n) =  3
h(n) =  7
f(n) =  10
2 1 6
4 5 *
7 3 8
g(n) =  3
h(n) =  7
f(n) =  10
2 * 1
4 8 6
7 5 3
g(n) =  4
h(n) =  6
f(n) =  10
4 2 6
1 * 8
7 5 3
g(n) =  4
h(n) =  6
f(n) =  10
1 4

7 * 8
5 2 3
g(n) =  8
h(n) =  7
f(n) =  15
1 4 6
7 2 8
5 3 *
g(n) =  8
h(n) =  7
f(n) =  15
2 1 6
7 * 3
8 4 5
g(n) =  8
h(n) =  7
f(n) =  15
4 2 1
7 * 6
5 8 3
g(n) =  8
h(n) =  7
f(n) =  15
4 2 1
7 8 6
5 3 *
g(n) =  8
h(n) =  7
f(n) =  15
* 6 8
2 1 3
4 7 5
g(n) =  8
h(n) =  7
f(n) =  15
2 6 8
1 * 3
4 7 5
g(n) =  9
h(n) =  6
f(n) =  15
4 1 6
8 2 *
7 5 3
g(n) =  8
h(n) =  7
f(n) =  15
4 6 8
1 2 3
7 5 *
g(n) =  8
h(n) =  7
f(n) =  15
4 6 8
1 * 2
7 5 3
g(n) =  8
h(n) =  7
f(n) =  15
* 2 6
4 7 8
5 1 3
g(n) =  8
h(n) =  7
f(n) =  15
4 2 6
5 7 8
* 1 3
g(n) =  8
h(n) =  7
f(n) =  15
4 2 *
7 8 6
5 1 3
g(n) =  8
h(n) =  7
f(n) =  15
4 2 6
7 8 3
5 1 *
g(n) =  8
h(n) =  7
f(n) =  15
4 2 *
7 1 6
5 3 8
g(n) =  8
h(n) =  7
f(n) =  15
4 2 6
7 * 1
5 3 8
g(n) =  7
h(n) =  8
f(n) =  15
2 6 8
5 1 *
4 7 3
g(n) =  7
h(n) =  8
f(n) =  15
2 1 6
5 7 *
4 3 8
g(n) =  7
h(n) =  8
f(n) =  15
2 * 1
5 8 6
4 7 3
g(n) =  7
h(n) =  8
f(n) =  15
2 1 6
5 8 3
4 * 7
g(n) =  7
h(n) =  8
f(n) =  15
2 6 8
7 1 *
5 4 3
g(n) =  

g(n) =  8
h(n) =  8
f(n) =  16
2 1 6
7 8 3
* 5 4
g(n) =  8
h(n) =  8
f(n) =  16
2 4 1
7 * 6
5 3 8
g(n) =  9
h(n) =  7
f(n) =  16
2 * 6
4 5 8
7 3 1
g(n) =  9
h(n) =  7
f(n) =  16
2 * 8
4 6 5
7 3 1
g(n) =  9
h(n) =  7
f(n) =  16
2 6 8
* 4 5
7 3 1
g(n) =  9
h(n) =  7
f(n) =  16
6 4 8
2 5 1
7 * 3
g(n) =  9
h(n) =  7
f(n) =  16
6 4 8
* 2 1
7 5 3
g(n) =  9
h(n) =  7
f(n) =  16
6 4 8
2 1 *
7 5 3
g(n) =  9
h(n) =  7
f(n) =  16
6 8 1
2 4 *
7 5 3
g(n) =  9
h(n) =  7
f(n) =  16
2 5 1
4 3 *
7 8 6
g(n) =  9
h(n) =  7
f(n) =  16
5 * 1
2 4 6
7 3 8
g(n) =  9
h(n) =  7
f(n) =  16
2 * 5
4 6 1
7 3 8
g(n) =  9
h(n) =  7
f(n) =  16
2 5 1
4 6 8
7 * 3
g(n) =  9
h(n) =  7
f(n) =  16
2 6 5
4 1 8
7 * 3
g(n) =  9
h(n) =  7
f(n) =  16
2 6 5
4 3 1
7 * 8
g(n) =  9
h(n) =  7
f(n) =  16
2 6 5
* 4 1
7 3 8
g(n) =  9
h(n) =  7
f(n) =  16
2 * 8
4 5 1
7 3 6
g(n) =  9
h(n) =  7
f(n) =  16
2 * 1
4 8 5
7 3 6
g(n) =  9
h(n) =  7
f(n) =  16
2 8 1
4 3 5
7 * 6
g(n) =  9
h(n) =  7
f(n) =  16
2 8 1
* 4 5
7 3 6
g(n) =  9
h(n) =  7


g(n) =  9
h(n) =  8
f(n) =  17
7 * 6
1 2 8
5 4 3
g(n) =  9
h(n) =  8
f(n) =  17
2 6 4
7 1 *
5 3 8
g(n) =  9
h(n) =  8
f(n) =  17
2 1 6
7 3 *
5 8 4
g(n) =  11
h(n) =  6
f(n) =  17
4 * 1
3 2 6
7 8 5
g(n) =  11
h(n) =  6
f(n) =  17
4 * 8
6 2 3
7 1 5
g(n) =  13
h(n) =  4
f(n) =  17
4 * 2
8 3 1
7 6 5
g(n) =  11
h(n) =  6
f(n) =  17
2 6 3
* 8 1
4 7 5
g(n) =  12
h(n) =  5
f(n) =  17
4 2 3
7 6 1
* 8 5
g(n) =  10
h(n) =  7
f(n) =  17
1 4 2
7 8 6
* 5 3
g(n) =  11
h(n) =  6
f(n) =  17
6 2 8
1 4 *
7 5 3
g(n) =  11
h(n) =  6
f(n) =  17
4 * 6
8 2 3
1 7 5
g(n) =  10
h(n) =  7
f(n) =  17
4 6 8
1 2 3
* 7 5
g(n) =  11
h(n) =  6
f(n) =  17
8 2 1
* 3 6
4 7 5
g(n) =  12
h(n) =  5
f(n) =  17
1 8 3
2 6 5
7 4 *
g(n) =  12
h(n) =  5
f(n) =  17
2 1 3
4 8 5
7 6 *
g(n) =  10
h(n) =  7
f(n) =  17
* 2 1
4 5 6
8 7 3
g(n) =  10
h(n) =  7
f(n) =  17
4 2 1
5 * 6
8 7 3
g(n) =  9
h(n) =  8
f(n) =  17
4 * 6
1 5 8
2 7 3
g(n) =  9
h(n) =  8
f(n) =  17
2 1 6
7 4 *
8 5 3
g(n) =  9
h(n) =  8
f(n) =  17
2 5 1
* 3 6
4 7 8
g(n) =

g(n) =  12
h(n) =  5
f(n) =  17
* 2 8
6 1 4
7 5 3
g(n) =  12
h(n) =  5
f(n) =  17
4 2 6
8 7 3
* 1 5
g(n) =  12
h(n) =  5
f(n) =  17
4 2 *
8 3 6
1 7 5
g(n) =  12
h(n) =  5
f(n) =  17
1 6 *
8 4 3
2 7 5
g(n) =  12
h(n) =  5
f(n) =  17
1 4 6
8 7 3
* 2 5
g(n) =  12
h(n) =  5
f(n) =  17
1 4 *
8 3 6
2 7 5
g(n) =  11
h(n) =  6
f(n) =  17
4 * 8
1 6 3
7 2 5
g(n) =  11
h(n) =  6
f(n) =  17
4 6 8
* 1 3
7 2 5
g(n) =  11
h(n) =  6
f(n) =  17
4 6 8
1 3 *
7 2 5
g(n) =  10
h(n) =  7
f(n) =  17
5 2 1
4 8 6
* 7 3
g(n) =  10
h(n) =  7
f(n) =  17
7 2 1
5 8 6
* 4 3
g(n) =  10
h(n) =  7
f(n) =  17
7 2 1
5 4 6
* 3 8
g(n) =  10
h(n) =  7
f(n) =  17
7 2 1
4 * 6
5 3 8
g(n) =  12
h(n) =  5
f(n) =  17
4 2 6
3 * 8
7 1 5
g(n) =  11
h(n) =  6
f(n) =  17
1 * 6
4 5 8
7 2 3
g(n) =  11
h(n) =  6
f(n) =  17
1 5 6
* 4 8
7 2 3
g(n) =  11
h(n) =  6
f(n) =  17
1 5 6
4 8 *
7 2 3
g(n) =  11
h(n) =  6
f(n) =  17
1 5 6
4 2 *
7 3 8
g(n) =  13
h(n) =  4
f(n) =  17
1 * 3
2 8 6
7 4 5
g(n) =  13
h(n) =  4
f(n) =  17
1 8 3
2 4 6
7 * 5


g(n) =  11
h(n) =  7
f(n) =  18
7 2 6
1 4 *
5 3 8
g(n) =  12
h(n) =  6
f(n) =  18
6 2 8
1 5 4
* 7 3
g(n) =  12
h(n) =  6
f(n) =  18
6 2 8
7 1 4
* 5 3
g(n) =  12
h(n) =  6
f(n) =  18
4 2 6
8 7 3
1 5 *
g(n) =  12
h(n) =  6
f(n) =  18
4 2 6
8 3 5
1 7 *
g(n) =  12
h(n) =  6
f(n) =  18
* 1 6
8 4 3
2 7 5
g(n) =  12
h(n) =  6
f(n) =  18
1 4 6
8 7 3
2 5 *
g(n) =  12
h(n) =  6
f(n) =  18
1 4 6
8 3 5
2 7 *
g(n) =  12
h(n) =  6
f(n) =  18
4 2 6
7 3 8
* 1 5
g(n) =  11
h(n) =  7
f(n) =  18
5 * 6
4 2 8
7 1 3
g(n) =  11
h(n) =  7
f(n) =  18
2 1 6
3 4 *
7 5 8
g(n) =  11
h(n) =  7
f(n) =  18
1 7 6
2 4 *
5 3 8
g(n) =  11
h(n) =  7
f(n) =  18
1 6 4
2 7 *
5 3 8
g(n) =  12
h(n) =  6
f(n) =  18
* 1 5
2 6 4
7 3 8
g(n) =  12
h(n) =  6
f(n) =  18
1 6 5
2 3 4
* 7 8
g(n) =  12
h(n) =  6
f(n) =  18
* 6 5
1 2 4
7 3 8
g(n) =  12
h(n) =  6
f(n) =  18
1 6 5
7 2 4
* 3 8
g(n) =  13
h(n) =  5
f(n) =  18
2 1 3
4 6 8
7 * 5
g(n) =  12
h(n) =  6
f(n) =  18
* 1 2
8 4 6
7 5 3
g(n) =  12
h(n) =  6
f(n) =  18
1 4 2
8 5 6
* 7 3


g(n) =  12
h(n) =  6
f(n) =  18
* 2 6
4 7 3
8 1 5
g(n) =  12
h(n) =  6
f(n) =  18
4 2 *
7 3 6
8 1 5
g(n) =  12
h(n) =  6
f(n) =  18
1 8 *
4 6 3
2 7 5
g(n) =  12
h(n) =  6
f(n) =  18
1 6 8
4 7 3
* 2 5
g(n) =  12
h(n) =  6
f(n) =  18
1 6 *
4 3 8
2 7 5
g(n) =  12
h(n) =  6
f(n) =  18
* 6 3
2 4 5
7 1 8
g(n) =  12
h(n) =  6
f(n) =  18
2 6 3
4 5 8
7 1 *
g(n) =  11
h(n) =  7
f(n) =  18
4 2 6
5 7 *
1 3 8
g(n) =  11
h(n) =  7
f(n) =  18
4 2 6
5 8 3
1 * 7
g(n) =  11
h(n) =  7
f(n) =  18
1 4 6
5 7 *
2 3 8
g(n) =  11
h(n) =  7
f(n) =  18
1 * 4
5 8 6
2 7 3
g(n) =  11
h(n) =  7
f(n) =  18
1 4 6
5 8 3
2 * 7
g(n) =  11
h(n) =  7
f(n) =  18
1 * 4
7 8 6
5 2 3
g(n) =  11
h(n) =  7
f(n) =  18
1 4 6
7 8 3
5 * 2
g(n) =  11
h(n) =  7
f(n) =  18
1 * 4
7 2 6
5 3 8
g(n) =  11
h(n) =  7
f(n) =  18
1 * 6
7 4 2
5 3 8
g(n) =  11
h(n) =  7
f(n) =  18
1 4 6
7 3 2
5 * 8
g(n) =  11
h(n) =  7
f(n) =  18
1 4 6
* 7 2
5 3 8
g(n) =  11
h(n) =  7
f(n) =  18
2 * 1
7 3 6
8 4 5
g(n) =  11
h(n) =  7
f(n) =  18
4 2 1
7 6 3
5 * 8


g(n) =  12
h(n) =  6
f(n) =  18
* 2 6
5 4 8
7 1 3
g(n) =  12
h(n) =  6
f(n) =  18
5 2 *
4 8 6
7 1 3
g(n) =  12
h(n) =  6
f(n) =  18
5 2 6
4 8 3
7 1 *
g(n) =  12
h(n) =  6
f(n) =  18
5 2 *
4 1 6
7 3 8
g(n) =  12
h(n) =  6
f(n) =  18
5 2 6
4 * 1
7 3 8
g(n) =  12
h(n) =  6
f(n) =  18
2 6 *
3 1 4
7 5 8
g(n) =  12
h(n) =  6
f(n) =  18
2 1 6
3 5 4
7 8 *
g(n) =  12
h(n) =  6
f(n) =  18
1 7 6
2 3 4
* 5 8
g(n) =  12
h(n) =  6
f(n) =  18
1 7 6
2 3 4
5 8 *
g(n) =  12
h(n) =  6
f(n) =  18
1 7 6
5 2 4
* 3 8
g(n) =  14
h(n) =  4
f(n) =  18
* 1 3
2 4 8
7 6 5
g(n) =  13
h(n) =  5
f(n) =  18
1 4 2
8 5 *
7 3 6
g(n) =  13
h(n) =  5
f(n) =  18
1 * 4
8 6 2
7 5 3
g(n) =  12
h(n) =  6
f(n) =  18
2 4 *
8 1 6
7 5 3
g(n) =  12
h(n) =  6
f(n) =  18
2 1 4
8 5 6
7 3 *
g(n) =  12
h(n) =  6
f(n) =  18
2 1 4
8 6 3
7 5 *
g(n) =  13
h(n) =  5
f(n) =  18
4 * 3
1 6 2
7 8 5
g(n) =  13
h(n) =  5
f(n) =  18
4 6 3
1 8 2
7 * 5
g(n) =  13
h(n) =  5
f(n) =  18
4 6 3
* 1 2
7 8 5
g(n) =  12
h(n) =  6
f(n) =  18
* 2 8
4 7 1
5 6 3


g(n) =  11
h(n) =  7
f(n) =  18
1 * 6
2 7 5
3 4 8
g(n) =  12
h(n) =  6
f(n) =  18
8 2 *
4 1 6
7 5 3
g(n) =  13
h(n) =  5
f(n) =  18
1 8 3
4 6 *
2 7 5
g(n) =  12
h(n) =  6
f(n) =  18
1 2 4
7 * 6
5 3 8
g(n) =  12
h(n) =  6
f(n) =  18
* 2 1
7 3 6
8 4 5
g(n) =  12
h(n) =  6
f(n) =  18
4 2 1
7 * 3
5 6 8
g(n) =  12
h(n) =  6
f(n) =  18
4 2 1
7 3 8
5 6 *
g(n) =  12
h(n) =  6
f(n) =  18
* 2 6
1 3 8
4 7 5
g(n) =  11
h(n) =  7
f(n) =  18
2 6 3
5 1 *
4 8 7
g(n) =  11
h(n) =  7
f(n) =  18
1 * 6
2 8 3
5 4 7
g(n) =  11
h(n) =  7
f(n) =  18
2 6 8
* 1 3
7 5 4
g(n) =  11
h(n) =  7
f(n) =  18
2 6 3
7 1 *
5 8 4
g(n) =  11
h(n) =  7
f(n) =  18
1 * 6
2 7 3
5 8 4
g(n) =  11
h(n) =  7
f(n) =  18
2 4 1
* 3 6
7 5 8
g(n) =  12
h(n) =  6
f(n) =  18
* 2 5
4 3 1
7 8 6
g(n) =  12
h(n) =  6
f(n) =  18
* 2 1
4 5 3
7 8 6
g(n) =  12
h(n) =  6
f(n) =  18
5 1 6
2 * 4
7 3 8
g(n) =  12
h(n) =  6
f(n) =  18
* 2 5
4 6 8
7 1 3
g(n) =  12
h(n) =  6
f(n) =  18
2 5 8
4 3 1
7 6 *
g(n) =  12
h(n) =  6
f(n) =  18
1 5 6
8 * 3
2 4 7


In [4]:
print('Search Cost:', search_cost)
print('')
display_solution(solution)

Search Cost: 4776

|SOLUTION|

g(n) =  0
h(n) =  7
f(n) =  7
2 1 6
4 * 8
7 5 3

g(n) =  1
h(n) =  7
f(n) =  8
2 1 6
4 8 *
7 5 3

g(n) =  2
h(n) =  7
f(n) =  9
2 1 *
4 8 6
7 5 3

g(n) =  3
h(n) =  7
f(n) =  10
2 * 1
4 8 6
7 5 3

g(n) =  4
h(n) =  7
f(n) =  11
2 8 1
4 * 6
7 5 3

g(n) =  5
h(n) =  7
f(n) =  12
2 8 1
4 6 *
7 5 3

g(n) =  6
h(n) =  7
f(n) =  13
2 8 1
4 6 3
7 5 *

g(n) =  7
h(n) =  6
f(n) =  13
2 8 1
4 6 3
7 * 5

g(n) =  8
h(n) =  5
f(n) =  13
2 8 1
4 * 3
7 6 5

g(n) =  9
h(n) =  5
f(n) =  14
2 8 1
* 4 3
7 6 5

g(n) =  10
h(n) =  5
f(n) =  15
* 8 1
2 4 3
7 6 5

g(n) =  11
h(n) =  5
f(n) =  16
8 * 1
2 4 3
7 6 5

g(n) =  12
h(n) =  5
f(n) =  17
8 1 *
2 4 3
7 6 5

g(n) =  13
h(n) =  4
f(n) =  17
8 1 3
2 4 *
7 6 5

g(n) =  14
h(n) =  3
f(n) =  17
8 1 3
2 * 4
7 6 5

g(n) =  15
h(n) =  3
f(n) =  18
8 1 3
* 2 4
7 6 5

g(n) =  16
h(n) =  2
f(n) =  18
* 1 3
8 2 4
7 6 5

g(n) =  17
h(n) =  1
f(n) =  18
1 * 3
8 2 4
7 6 5

g(n) =  18
h(n) =  0
f(n) =  18
1 2 3
8 * 4
7 6 5



In [5]:
heuristic_function = get_manhattan_distance_heuristic_3x3
solution, search_cost = solve_8_puzzle_with_a_star(start_configuration, heuristic_function, is_manhattan=True)

g(n) =  0
h(n) =  12
f(n) =  12
2 1 6
4 * 8
7 5 3
g(n) =  1
h(n) =  11
f(n) =  12
P(n) =  11
2 1 6
* 4 8
7 5 3
g(n) =  1
h(n) =  11
f(n) =  12
P(n) =  11
2 1 6
4 8 *
7 5 3
g(n) =  2
h(n) =  10
f(n) =  12
P(n) =  10
2 1 *
4 8 6
7 5 3
g(n) =  2
h(n) =  10
f(n) =  12
P(n) =  10
2 1 6
4 8 3
7 5 *
g(n) =  3
h(n) =  9
f(n) =  12
P(n) =  9
2 1 6
4 8 3
7 * 5
g(n) =  1
h(n) =  13
f(n) =  14
P(n) =  13
2 * 6
4 1 8
7 5 3
g(n) =  1
h(n) =  13
f(n) =  14
P(n) =  13
2 1 6
4 5 8
7 * 3
g(n) =  2
h(n) =  12
f(n) =  14
P(n) =  12
* 1 6
2 4 8
7 5 3
g(n) =  2
h(n) =  12
f(n) =  14
P(n) =  12
2 1 6
7 4 8
* 5 3
g(n) =  3
h(n) =  11
f(n) =  14
P(n) =  11
2 * 1
4 8 6
7 5 3
g(n) =  4
h(n) =  10
f(n) =  14
P(n) =  10
2 1 6
4 * 3
7 8 5
g(n) =  4
h(n) =  10
f(n) =  14
P(n) =  10
2 1 6
4 8 3
* 7 5
g(n) =  2
h(n) =  12
f(n) =  14
P(n) =  12
* 2 6
4 1 8
7 5 3
g(n) =  2
h(n) =  12
f(n) =  14
P(n) =  12
2 6 *
4 1 8
7 5 3
g(n) =  3
h(n) =  11
f(n) =  14
P(n) =  11
1 * 6
2 4 8
7 5 3
g(n) =  4
h(n) =  10
f(n) =  14
P(n) 

h(n) =  13
f(n) =  18
P(n) =  13
2 1 6
4 3 5
7 * 8
g(n) =  7
h(n) =  11
f(n) =  18
P(n) =  11
1 * 6
2 8 3
4 7 5
g(n) =  5
h(n) =  13
f(n) =  18
P(n) =  13
2 * 8
4 6 1
7 5 3
g(n) =  5
h(n) =  13
f(n) =  18
P(n) =  13
2 6 8
* 4 1
7 5 3
g(n) =  7
h(n) =  11
f(n) =  18
P(n) =  11
8 * 1
2 4 6
7 5 3
g(n) =  8
h(n) =  10
f(n) =  18
P(n) =  10
2 3 1
4 * 6
7 8 5
g(n) =  8
h(n) =  10
f(n) =  18
P(n) =  10
* 2 1
4 3 6
7 8 5
g(n) =  8
h(n) =  10
f(n) =  18
P(n) =  10
* 2 6
8 1 3
4 7 5
g(n) =  8
h(n) =  10
f(n) =  18
P(n) =  10
2 6 *
8 1 3
4 7 5
g(n) =  8
h(n) =  10
f(n) =  18
P(n) =  10
2 1 6
8 7 3
* 4 5
g(n) =  8
h(n) =  10
f(n) =  18
P(n) =  10
2 1 *
8 3 6
4 7 5
g(n) =  6
h(n) =  12
f(n) =  18
P(n) =  12
4 6 *
1 2 8
7 5 3
g(n) =  8
h(n) =  10
f(n) =  18
P(n) =  10
* 4 1
8 2 6
7 5 3
g(n) =  8
h(n) =  10
f(n) =  18
P(n) =  10
4 1 *
8 2 6
7 5 3
g(n) =  9
h(n) =  9
f(n) =  18
P(n) =  9
1 4 6
* 2 3
7 8 5
g(n) =  9
h(n) =  9
f(n) =  18
P(n) =  9
2 * 3
4 6 1
7 8 5
g(n) =  9
h(n) =  9
f(n) =  18
P(n) = 

In [6]:
print('Search Cost:', search_cost)
print('')
display_solution(solution)

Search Cost: 615

|SOLUTION|

g(n) =  0
h(n) =  12
f(n) =  12
2 1 6
4 * 8
7 5 3

g(n) =  1
h(n) =  11
f(n) =  12
P(n) =  11
2 1 6
4 8 *
7 5 3

g(n) =  2
h(n) =  10
f(n) =  12
P(n) =  10
2 1 *
4 8 6
7 5 3

g(n) =  3
h(n) =  11
f(n) =  14
P(n) =  11
2 * 1
4 8 6
7 5 3

g(n) =  4
h(n) =  12
f(n) =  16
P(n) =  12
2 8 1
4 * 6
7 5 3

g(n) =  5
h(n) =  11
f(n) =  16
P(n) =  11
2 8 1
4 6 *
7 5 3

g(n) =  6
h(n) =  10
f(n) =  16
P(n) =  10
2 8 1
4 6 3
7 5 *

g(n) =  7
h(n) =  9
f(n) =  16
P(n) =  9
2 8 1
4 6 3
7 * 5

g(n) =  8
h(n) =  8
f(n) =  16
P(n) =  8
2 8 1
4 * 3
7 6 5

g(n) =  9
h(n) =  7
f(n) =  16
P(n) =  7
2 8 1
* 4 3
7 6 5

g(n) =  10
h(n) =  8
f(n) =  18
P(n) =  8
* 8 1
2 4 3
7 6 5

g(n) =  11
h(n) =  7
f(n) =  18
P(n) =  7
8 * 1
2 4 3
7 6 5

g(n) =  12
h(n) =  6
f(n) =  18
P(n) =  6
8 1 *
2 4 3
7 6 5

g(n) =  13
h(n) =  5
f(n) =  18
P(n) =  5
8 1 3
2 4 *
7 6 5

g(n) =  14
h(n) =  4
f(n) =  18
P(n) =  4
8 1 3
2 * 4
7 6 5

g(n) =  15
h(n) =  3
f(n) =  18
P(n) =  3
8 1 3
* 2 4
7 6 5

g

In [7]:
heuristic_function = get_nilsson_sequence_score
solution, search_cost = solve_8_puzzle_with_a_star(start_configuration, heuristic_function, is_nilsson=True)

g(n) =  0
h(n) =  (60, 12, 16)
f(n) =  60
2 1 6
4 * 8
7 5 3
g(n) =  1
h(n) =  56
f(n) =  57
P(n) =  11
S(n) =  15
2 1 6
* 4 8
7 5 3
g(n) =  1
h(n) =  56
f(n) =  57
P(n) =  11
S(n) =  15
2 1 6
4 8 *
7 5 3
g(n) =  2
h(n) =  55
f(n) =  57
P(n) =  10
S(n) =  15
2 1 *
4 8 6
7 5 3
g(n) =  2
h(n) =  55
f(n) =  57
P(n) =  10
S(n) =  15
2 1 6
4 8 3
7 5 *
g(n) =  3
h(n) =  54
f(n) =  57
P(n) =  9
S(n) =  15
2 1 6
4 8 3
7 * 5
g(n) =  1
h(n) =  58
f(n) =  59
P(n) =  13
S(n) =  15
2 * 6
4 1 8
7 5 3
g(n) =  1
h(n) =  58
f(n) =  59
P(n) =  13
S(n) =  15
2 1 6
4 5 8
7 * 3
g(n) =  2
h(n) =  57
f(n) =  59
P(n) =  12
S(n) =  15
* 1 6
2 4 8
7 5 3
g(n) =  2
h(n) =  57
f(n) =  59
P(n) =  12
S(n) =  15
2 1 6
7 4 8
* 5 3
g(n) =  3
h(n) =  56
f(n) =  59
P(n) =  11
S(n) =  15
2 * 1
4 8 6
7 5 3
g(n) =  4
h(n) =  54
f(n) =  58
P(n) =  12
S(n) =  14
2 8 1
4 * 6
7 5 3
g(n) =  5
h(n) =  50
f(n) =  55
P(n) =  11
S(n) =  13
2 8 1
* 4 6
7 5 3
g(n) =  5
h(n) =  50
f(n) =  55
P(n) =  11
S(n) =  13
2 8 1
4 6 *
7 5 3
g(n) 

In [8]:
print('Search Cost:', search_cost)
print('')
display_solution(solution)

Search Cost: 89

|SOLUTION|

g(n) =  0
h(n) =  (60, 12, 16)
f(n) =  60
2 1 6
4 * 8
7 5 3

g(n) =  1
h(n) =  56
f(n) =  57
P(n) =  11
S(n) =  15
2 1 6
4 8 *
7 5 3

g(n) =  2
h(n) =  55
f(n) =  57
P(n) =  10
S(n) =  15
2 1 *
4 8 6
7 5 3

g(n) =  3
h(n) =  56
f(n) =  59
P(n) =  11
S(n) =  15
2 * 1
4 8 6
7 5 3

g(n) =  4
h(n) =  54
f(n) =  58
P(n) =  12
S(n) =  14
2 8 1
4 * 6
7 5 3

g(n) =  5
h(n) =  50
f(n) =  55
P(n) =  11
S(n) =  13
2 8 1
4 6 *
7 5 3

g(n) =  6
h(n) =  49
f(n) =  55
P(n) =  10
S(n) =  13
2 8 1
4 6 3
7 5 *

g(n) =  7
h(n) =  48
f(n) =  55
P(n) =  9
S(n) =  13
2 8 1
4 6 3
7 * 5

g(n) =  8
h(n) =  38
f(n) =  46
P(n) =  8
S(n) =  10
2 8 1
4 * 3
7 6 5

g(n) =  9
h(n) =  34
f(n) =  43
P(n) =  7
S(n) =  9
2 8 1
* 4 3
7 6 5

g(n) =  10
h(n) =  35
f(n) =  45
P(n) =  8
S(n) =  9
* 8 1
2 4 3
7 6 5

g(n) =  11
h(n) =  40
f(n) =  51
P(n) =  7
S(n) =  11
8 * 1
2 4 3
7 6 5

g(n) =  12
h(n) =  33
f(n) =  45
P(n) =  6
S(n) =  9
8 1 *
2 4 3
7 6 5

g(n) =  13
h(n) =  32
f(n) =  45
P(n) = 